In [46]:
import tensorflow as tf

from tensorflow import keras
from keras import layers

In [45]:
dim1, dim2 = 32, 32 # change me :)

img_dim = (dim1, dim2, 3) # change me :3

model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=img_dim,
    alpha=1.0, # control width of network layers (less than 1.0 means decreases no. filters, more than 1.0 means increases no. filters)
    include_top=True, # include fully connected layer
    weights=None, # no image weights
    input_tensor=None, # none
    pooling=None, # maybe we can edit this?
    classes=5, # default was 1000, maybe we dont need to replace last layer with dense layer of 5 if this is already 5?
    classifier_activation='softmax' # default
)

In [44]:
model.summary()

Model: "mobilenetv2_1.00_32"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 16, 16, 32)   864         ['input_10[0][0]']               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 16, 16, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 16, 16, 32)   0           ['bn_Conv1[0][0]']               
                                                                                

In [43]:
model_popped = model.layers[-1].output

outputs = layers.Dense(5)(model_popped)
new_model = keras.Model(inputs=model.input, outputs=outputs)

new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 16, 16, 32)   864         ['input_10[0][0]']               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 16, 16, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 16, 16, 32)   0           ['bn_Conv1[0][0]']               
                                                                                            

In [42]:
initial_learning_rate = 0.01 # learning rate they want

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay( # maybe we do not need this?
    initial_learning_rate,
    decay_steps=100000, # defualt values
    decay_rate=0.96, # defualt values
    staircase=True # defualt values
)

optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_schedule, 
    momentum=0.0, 
    nesterov=False, 
    name="SGD"
)